# Import Library

In [1]:
# Import Library
import pandas as pd
import os
from dotenv import load_dotenv
import pymongo
from pymongo import MongoClient
from langchain_openai import ChatOpenAI
from langchain_mongodb import MongoDBAtlasVectorSearch
from langchain.chains import RetrievalQA
from langchain import PromptTemplate
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.chat_models import ChatOpenAI
import re

# Set Up

In [2]:
# Load semua environment
load_dotenv()
MONGODB_URI = os.getenv("MONGODB_URI")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
DEEPSEEK_API_NEW=os.getenv("DEEPSEEK_API_NEW")

In [3]:
# Embeding menggunakan gemini
embeddings = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001",
    google_api_key=GOOGLE_API_KEY,
)

In [4]:
# MongoDB Connection ke
client = MongoClient(MONGODB_URI)
collection = client['finalproject_db']['faq_tb']

In [5]:
# Vector Store Configuration
vector_store = MongoDBAtlasVectorSearch(
    collection=collection,
    embedding=embeddings,
    index_name='vector_index',
)

In [6]:
# Prompt Template
PROFESSIONAL_PROMPT = PromptTemplate(
    input_variables=["context", "question"],
    template="""
    Anda adalah Asisten Digital Resmi BPJS Kesehatan dan BPJS Ketenagakerjaan.
    Tugas Anda sebagai Customer Service adalah menjawab pertanyaan peserta berdasarkan data FAQ resmi yang telah disediakan.

    Tanggung jawab utama:
    
    Memberikan jawaban yang akurat dan informatif sesuai dengan kategori FAQ BPJS Kesehatan dan BPJS Ketenagakerjaan.
    Menyampaikan informasi dengan bahasa yang profesional, jelas, dan mudah dipahami oleh seluruh lapisan masyarakat.
    Memberikan panduan teknis atau prosedur (jika diperlukan) dalam format langkah-langkah yang berurutan atau poin-poin.
    Menanggapi pertanyaan atau peserta dengan solusi yang relevan dari konteks FAQ yang tersedia.

        Aturan jawaban:
        
    Jawab hanya berdasarkan informasi yang tersedia dalam konteks FAQ resmi yang diberikan.
    Jika pertanyaan tidak relevan atau tidak tercakup dalam FAQ, jawab dengan sopan: "Mohon maaf, saya tidak memiliki informasi mengenai hal tersebut. Saya hanya dapat menjawab pertanyaan seputar layanan BPJS Kesehatan dan BPJS Ketenagakerjaan."
    Jangan mengarang jawaban atau memberikan informasi di luar dokumen sumber.
    Jangan mencantumkan link eksternal atau informasi promosi di luar konteks.
    Gunakan format poin atau numerik bila menjelaskan prosedur atau langkah-langkah.

    Konteks resmi (dari FAQ):
    {context}

    Pertanyaan Peserta: {question}

    Jawaban Profesional:
    """
)

In [7]:
# Config model untuk Deepseek
llm_deepseek = ChatOpenAI(
    model="deepseek-chat",  
    openai_api_key=DEEPSEEK_API_NEW,  
    base_url="https://api.deepseek.com/v1",  
    temperature=0,
    max_tokens=500
)

C:\Users\THINKPAD\AppData\Local\Temp\ipykernel_7112\2099205823.py:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm_deepseek = ChatOpenAI(


In [8]:
# Retrieval Chain
qa = RetrievalQA.from_chain_type(
    llm=llm_deepseek,
    chain_type="stuff",
    retriever=vector_store.as_retriever(
        search_type="similarity",
        search_kwargs={
            "k": 5,
            "score_threshold": 0.85
        }
    ),
    chain_type_kwargs={"prompt": PROFESSIONAL_PROMPT},
)

In [9]:
# Ambil data
faq_df = pd.read_csv('../data/final_dataset_clean.csv')
def answer_from_doc(question):
    """Fungsi untuk mencocokkan pertanyaan dengan data yang sama persis dan mengambil jawaban"""
    row = faq_df[faq_df['question'] == question]
    if not row.empty:
        return row.iloc[-1]['answer']
    else:
        return "Tidak ada jawaban."

In [10]:
def clean_answer(text):
    """Hilangkan tanda bold/italic Markdown"""
    text = re.sub(r'\*\*(.*?)\*\*', r'\1', text)
    text = re.sub(r'\*(.*?)\*', r'\1', text)
    cleaned_text = re.sub(r'\n{3,}', '\n\n', text)
    return cleaned_text.strip()
    

# Evaluation

## Test Case 1 Pertanyaan Dataset

In [18]:
# UJI COBA 1: Pertanyaan yang Seharusnya Bisa Dijawab karena berasal dari dataset
questions = [
# Mobile
"Bagaimana cara menginstall aplikasi Jamsostek Mobile (JMO)?",
"Bagaimana cara mengajukan klaim JHT melalui JMO jika memiliki lebih dari 1 kartu?",
"Promo apa saja yang tersedia pada aplikasi Jamsostek Mobile (JMO)?",
"Jika anggota baru di JMO, apakah harus mendaftar di MotionPay?",
"Apakah itu Promo?",
"Apakah itu MotionPay?",
"Apakah sandi/PIN harus diubah secara berkala di kedua aplikasi?",
"Promo apa saja yang tersedia pada aplikasi Jamsostek Mobile (JMO)?",

# Ketenagakerjaan
"Apa itu BPJS Ketenagakerjaan?",
"Apa saja program BPJS Ketenagakerjaan?",
"Siapakah yang dimaksud peserta BPJS Ketenagakerjaan?",
"Apakah yang dimaksud dengan PHK?",
"Apakah pegawai BHL (Buruh Harian Lepas) dapat mengikuti program JKP?",
"Bagaimana caranya badan usaha/pemberi kerja dapat menggunakan layanan Payment Reminder System(PRS)?",
"Peserta BPU sudah melakukan pembayaran iuran namun transaksi pembayaran iuran belum merubah masa perlindungan/saldo JHT belum bertambah, apakah yang harus dilakukan?",
"Apakah aplikasi Electronic Payment System (EPS)?",
"Apa perbedaan program Jaminan Hari Tua (JHT) dan Jaminan Pensiun (JP)?",

# Kesehatan
"Apa saja pelayanan BPJS Kesehatan yang dapat dimanfaatkan ?",
"Bagaimana Mendaftarkan Bayi Baru Lahir ?",
"Dimana kontak kantor BPJS Kesehatan ?",
"Apa itu GRC ?",
"Cara Membayar Iuran",
"Apa saja pelayanan BPJS Kesehatan yang dapat dimanfaatkan ?",
"Kapan pembayaran iuran paling lambat ?",
"Bagaimana komitmen BPJS Kesehatan terkait gratifikasi ?"
]

for q in questions:
    print(f"\n{'='*50}")
    print(f"Pertanyaan: {q}")
    result = qa.invoke({"query": q})
    print(clean_answer(result['result']))
    print(f"\nACTUAL DATA: {answer_from_doc(q)}")
    print(f"{'='*50}")


Pertanyaan: Bagaimana cara menginstall aplikasi Jamsostek Mobile (JMO)?
Berikut adalah langkah-langkah untuk menginstal aplikasi Jamsostek Mobile (JMO):  

1. Buka Play Store (Android) atau App Store (iOS) di perangkat Anda.  
2. Cari "Jamsostek Mobile" di kolom pencarian.  
3. Pilih aplikasi resmi yang diterbitkan oleh BPJS Ketenagakerjaan.  
4. Klik "Install" untuk mengunduh dan menginstal aplikasi.  
5. Buka aplikasi JMO setelah proses instalasi selesai.  
6. Lakukan registrasi atau login menggunakan akun yang terdaftar di BPJS Ketenagakerjaan.  

Jika mengalami kendala, pastikan perangkat Anda memenuhi persyaratan sistem dan terhubung dengan jaringan internet yang stabil.  

Mohon maaf jika pertanyaan Anda tidak tercakup dalam FAQ di atas. Saya hanya dapat menjawab pertanyaan seputar layanan BPJS Kesehatan dan BPJS Ketenagakerjaan.

ACTUAL DATA: Aplikasi Jamsostek Mobile (JMO) dapat diunduh dan diinstall melalui Playstore/ Appstore bagi pengguna smartphone .

Pertanyaan: Bagaimana

Total pertanyaan sesuai dataset : 25

Pertanyaan terjawab             : 18

Pertanyaan tidak terjawab       : 7

List pertanyaan tidak terjawab :

1. Promo apa saja yang tersedia pada aplikasi Jamsostek Mobile (JMO)? (Ketenagakerjaan)
2. Jika anggota baru di JMO, apakah harus mendaftar di MotionPay? (Ketenagakerjaan)
3. Apakah itu Promo? (Ketenagakerjaan)
4. Apakah itu MotionPay? (Ketenagakerjaan)
5. Promo apa saja yang tersedia pada aplikasi Jamsostek Mobile (JMO)? (Ketenagakerjaan)
6. Apa itu GRC ? (Kesehatan)
7. Bagaimana komitmen BPJS Kesehatan terkait gratifikasi ? (Kesehatan)

Jika dibandingkan dengan size embeding 300, overlap 50, treshold 70%, model deepseek size embeding 500, overlap 100 dan treshold 85% menjawab lebih banyak, dan lebih detail

**Pertanyaan : Bagaimana cara menginstall aplikasi Jamsostek Mobile (JMO)?**

**Jawaban Deepseek Size Embeding 500:**

Berikut adalah langkah-langkah untuk menginstal aplikasi Jamsostek Mobile (JMO):  

1. Buka Play Store (Android) atau App Store (iOS) di perangkat Anda.  
2. Cari "Jamsostek Mobile" di kolom pencarian.  
3. Pilih aplikasi resmi yang diterbitkan oleh BPJS Ketenagakerjaan.  
4. Klik "Install" untuk mengunduh dan menginstal aplikasi.  
5. Buka aplikasi JMO setelah proses instalasi selesai.  
6. Lakukan registrasi atau login menggunakan akun yang terdaftar di BPJS Ketenagakerjaan.  

Jika mengalami kendala, pastikan perangkat Anda memenuhi persyaratan sistem dan terhubung dengan jaringan internet yang stabil.  

Mohon maaf jika pertanyaan Anda tidak tercakup dalam FAQ di atas. Saya hanya dapat menjawab pertanyaan seputar layanan BPJS Kesehatan dan BPJS Ketenagakerjaan.

**Jawaban Deepseek Size Embeding 300:**

Untuk menginstal aplikasi Jamsostek Mobile (JMO), berikut langkah-langkahnya:  

1. Buka Google Play Store (Android) atau App Store (iOS) di perangkat Anda.  
2. Cari "Jamsostek Mobile" di kolom pencarian.  
3. Pilih aplikasi resmi yang diterbitkan oleh BPJS Ketenagakerjaan.  
4. Klik "Instal" dan tunggu proses pengunduhan selesai.  
5. Buka aplikasi dan ikuti panduan registrasi/login sesuai kebutuhan.  

Pastikan perangkat Anda terhubung dengan internet dan memiliki ruang penyimpanan yang cukup.  

Jika mengalami kendala, pastikan aplikasi yang diunduh adalah versi terbaru dan sesuai dengan sistem operasi perangkat Anda.



## Test Case 2 Pertanyaan Diluar Dataset

In [19]:
# UJI COBA 2: Pertanyaan di Luar Konteks
questions = [
    "Apa itu Allo Bank?",
    "Bagaimana cara mengajukan KPR?",
    "Siapa penemu listrik?",
    "Apa saja manfaat dari olahraga?",
    "Bagaimana cara membuat kue?",
    "Jelaskan apa itu Quantum Computing?",
    "Apa yang dimaksud dengan Machine Learning?",
    "Gimana cara buat foto HD dengan AI?",
    "Apa itu teknologi blockchain?",
    "Apa itu teknologi quantum?",
    "Apa kopi paling enak saat ini?",
    "Sebutkan Sepatu lari paling enak",
    "Kapan Indonesia Merdeka?",
    "Dimana kantor urusan agama terdekat",
    "Jelaskan tentang skema piala dunia?",
    "Siapa pencetak gol terbanyak piala dunia 2022?",
    "Jumlah total trofi Liverpool berapa?",
    "Hitung luas Kota Surabaya",
    "Dimana tempat syuting film laskar Pelangi?",
    "Dimana bom atom amerika saat perang dunia kedua dibuat?",
    "Bagaimana cara membedakan biji kopi?",
    "Jembatan merah ada Dimana?",
    "Siapa presiden Indonesia pertama?",
    "Siapa penemu alat musik gitar?",
    "Hitung luas lapangan bola."
    ]

for q in questions:
    print(f"\n{'='*50}")
    print(f"Pertanyaan: {q}")
    result = qa.invoke({"query": q})
    print(clean_answer(result['result']))
    print(f"\nACTUAL DATA: {answer_from_doc(q)}")
    print(f"{'='*50}")


Pertanyaan: Apa itu Allo Bank?
Mohon maaf, saya tidak memiliki informasi mengenai hal tersebut. Saya hanya dapat menjawab pertanyaan seputar layanan BPJS Kesehatan dan BPJS Ketenagakerjaan.  

Silakan bertanya terkait program, manfaat, prosedur, atau kebijakan BPJS Kesehatan/BPJS Ketenagakerjaan agar saya dapat membantu Anda dengan informasi resmi yang akurat.

ACTUAL DATA: Tidak ada jawaban.

Pertanyaan: Bagaimana cara mengajukan KPR?
Mohon maaf, saya tidak memiliki informasi mengenai hal tersebut. Saya hanya dapat menjawab pertanyaan seputar layanan BPJS Kesehatan dan BPJS Ketenagakerjaan.  

Untuk pertanyaan terkait KPR, Anda dapat menghubungi pihak bank atau lembaga keuangan terkait yang menyediakan layanan tersebut.  

Jika Anda memiliki pertanyaan seputar BPJS Kesehatan atau BPJS Ketenagakerjaan, saya dengan senang hati akan membantu.

ACTUAL DATA: Tidak ada jawaban.

Pertanyaan: Siapa penemu listrik?
Mohon maaf, saya tidak memiliki informasi mengenai hal tersebut. Saya hanya da

Total pertanyaan diluat dataset : 25

Pertanyaan terjawab             : 0

Pertanyaan tidak terjawab       : 25

Semua pertanyaan tidak dijawab oleh model, hal ini bisa terjadi karena set treshold similarity sebesar 85%, yang cukup tinggi untuk membedakan kemiripan data. Peforma model untuk tidak menjawab pertanyaan diluar konteks sangat bagus

## Test Case 3 Pertanyaan Tidak Lengkap/Typo

In [20]:
# UJI COBA 3: Pertanyaan Masih dalam Konteks namun terdapat kesalahan input
questions = [
   "Aph itu BPJS Ketenagakerajaan?",
    "Apa sajak program BPJS Ketenagakerjaan?",
    "Siapakah yang dimaksud peserta BPJS Ketenagakerajaan?",
    "Apakh yang dimaksud dengan PHK?",
    "Apakah pegawai BHL (Burung Harian Lepas) dapat mengikuti program JKP?",
    "Peserta BPU sudah melakukan pembayaran iuran namun transaksi pembayaran iuran belum merubah masa perlindungan/saldo JHT belum bertambah, apakah yang harus dilakukan?", 
    "Gimana jika lupa PIN EPS?",
    "Apakah itu Perumahan Pekerja?",
    "Dokumen apa sajakah persyaratan mengajukan JHT bagi PMI yang berpindah menjadi Warga Negara Asing?",
    "Kapan BSU tahun 2025 dibagikan?",
    "Dimana obat batuk pertama kali ditemukan",
    "Jika flu sebaiknya minum obat herbal apa",
    "Sopo peserta BPJS ketenagakerjaan",
    "Sejak tanggal berapa pembayaran iuran paling lambat ?",
    "Jika anggota baru di JMO, apakah harus mendaftar di MotionIme?",
    "Apakah itu ShoopePay?",
    "Apa perbedaan program Jaminan Dari Tua (JHT) dan Jaminan Pensiun (JP)?",
    "Gimana pembayaran iuran?",
    "Tanggal pembayaran iuran paling lambat ?",
    "Promo apa sajak yang tersedia pada aplikasi Jamsostek Mobile (JMO)?"
]

for q in questions:
    print(f"\n{'='*50}")
    print(f"Pertanyaan: {q}")
    result = qa.invoke({"query": q})
    print(clean_answer(result['result']))
    print(f"\nACTUAL DATA: {answer_from_doc(q)}")
    print(f"{'='*50}")


Pertanyaan: Aph itu BPJS Ketenagakerajaan?
BPJS Ketenagakerjaan adalah program perlindungan sosial yang diselenggarakan oleh pemerintah Indonesia untuk memberikan jaminan sosial kepada tenaga kerja. Program ini mencakup beberapa manfaat, antara lain:  

1. Jaminan Kecelakaan Kerja (JKK) – Memberikan perlindungan atas risiko kecelakaan kerja atau penyakit akibat kerja.  
2. Jaminan Hari Tua (JHT) – Memberikan manfaat berupa tabungan hari tua yang dapat dicairkan saat peserta memenuhi persyaratan tertentu.  
3. Jaminan Pensiun (JP) – Memberikan manfaat pensiun bagi peserta yang telah mencapai usia pensiun atau mengalami cacat total tetap.  
4. Jaminan Kematian (JKm) – Memberikan santunan kematian kepada ahli waris peserta.  

BPJS Ketenagakerjaan wajib bagi pekerja formal (karyawan) dan sukarela bagi pekerja informal.  

Jika Anda membutuhkan informasi lebih lanjut, silakan hubungi layanan pelanggan BPJS Ketenagakerjaan atau kunjungi kantor terdekat.

ACTUAL DATA: Tidak ada jawaban.

Pe

Total pertanyaan salah tidak baku/typo : 20

Pertanyaan terjawab             : 12

Pertanyaan tidak terjawab       : 8

Pertanyaan tidak terjawab
1. Apakah pegawai BHL (Burung Harian Lepas) dapat mengikuti program JKP?
2. Apakah itu Perumahan Pekerja?
3. Kapan BSU tahun 2025 dibagikan?
4. Dimana obat batuk pertama kali ditemukan
5. Jika flu sebaiknya minum obat herbal apa
6. Jika anggota baru di JMO, apakah harus mendaftar di MotionIme?
7. Apakah itu ShoopePay?
8. Promo apa sajak yang tersedia pada aplikasi Jamsostek Mobile (JMO)?

Analisa:

Model mampu menjawab penggunaan bahasa tidak baku seperti Sopo peserta BPJS ketenagakerjaan, namun karena ragu, model juga memberikan permintaan maaf, dan jawaban yang mirip seperti. Mohon maaf, saya tidak memiliki informasi mengenai hal tersebut. Saya hanya dapat menjawab pertanyaan seputar layanan BPJS Kesehatan dan BPJS Ketenagakerjaan. Jika Anda ingin mengetahui siapa saja yang termasuk sebagai peserta BPJS Ketenagakerjaan, berikut informasinya......
Model sedikit lebih baik dari pada dengan embeding 300

**Embeding Size 500 :**

Mohon maaf, saya tidak memiliki informasi mengenai hal tersebut. Saya hanya dapat menjawab pertanyaan seputar layanan BPJS Kesehatan dan BPJS Ketenagakerjaan.  

Jika Anda ingin mengetahui siapa saja yang termasuk sebagai peserta BPJS Ketenagakerjaan, berikut informasinya:  

Peserta BPJS Ketenagakerjaan meliputi:  
1. Pekerja penerima upah, seperti:  
   - Pegawai swasta  
   - Pegawai pemerintah  
   - Pekerja di perusahaan  
2. Pekerja bukan penerima upah, seperti:  
   - Pekerja mandiri (freelancer, wiraswasta)  
   - Pekerja di sektor informal  
3. Pekerja migran Indonesia yang bekerja di luar negeri.  

Jika Anda membutuhkan informasi lebih lanjut, silakan tanyakan pertanyaan yang lebih spesifik terkait BPJS Ketenagakerjaan.

**Embeding Size 300 :**

Maaf, saya tidak menemukan informasi terkait pertanyaan "Sopo peserta BPJS ketenagakerjaan" dalam dokumen resmi BPJS Ketenagakerjaan.  

Untuk informasi peserta BPJS Ketenagakerjaan, Anda dapat mengunjungi website resmi BPJS Ketenagakerjaan atau menghubungi layanan pelanggan mereka.  

Mohon pastikan pertanyaan menggunakan bahasa yang jelas dan sesuai konteks layanan BPJS.

# Kesimpulan

Deepseek dengan Embeding Size 500, overlap 100, dan treshold 85% lebih dari jumlah menjawab dan detail, serta reasoning pertanyaan tidak baku/typo. Presentase model Embeding Size 500 menjawab dengan benar sebesar 78%.

1. Chatbot mampu memberikan jawaban yang relevan sesuai dengan dataset. Fokus utama adalah untuk meningkatkan model pada pertanyaan dengan kategori E-CHANNEL, sub-kategori JMO, pertanyaan dengan singkatan dan memiliki deskripsi singkat kurang dikenali seperti Apa itu Promo ?, Pertanyaan singkatan yang mengandung arti lain dipahami oleh BOT sebagai pertanyaan diluar konteks.

2. Chatbot memiliki kinerja yang bagus untuk tidak menjawab pertanyaan diluar konteks, hal ini dikarenakan treshold yang tinggi dan prompt untuk tidak menjawab pertanyaan tidak relevan.

3. Chatbot mampu untuk menjawab pertanyaan typo yang tidak merubah arti, bahasa indonesia yang tidak baku, seperti pada pertanyaan Sopo peserta BPJS ketenagakerjaan. Namun chatbot gagal menjawab pertanyaan typo yang memiliki arti lain seperti Apakah pegawai BHL (Burung Harian Lepas) dapat mengikuti program JKP?.
